In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import numpy as np
from medusa.model.medusa_model import MedusaModel
from medusa.model.kv_cache import *
from medusa.model.utils import *
from medusa.model.medusa_choices import *
from copy import deepcopy
from tqdm import tqdm
import matplotlib.pyplot as plt

/home/takuro.fujii/.pyenv/versions/3.11.0/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using `is_flash_attn_available` is deprecated and will be removed in v4.38. Please use `is_flash_attn_2_available` instead.
Using `is_flash_attn_available` is deprecated and will be removed in v4.38. Please use `is_flash_attn_2_available` instead.


In [2]:
# config
model_name = 'FasterDecoding/medusa-vicuna-7b-v1.3'
load_in_8bit = True
load_in_4bit = False

# モデル、トークナイザ
model = MedusaModel.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    load_in_8bit=load_in_8bit,
    load_in_4bit=load_in_4bit,
)
tokenizer = model.get_tokenizer()

medusa_choices = mc_sim_7b_63

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.47s/it]
Some weights of MedusaModelLlama were not initialized from the model checkpoint at lmsys/vicuna-7b-v1.3 and are newly initialized: ['medusa_head.2.1.weight', 'medusa_head.4.1.weight', 'medusa_head.2.0.linear.bias', 'medusa_head.4.0.linear.weight', 'medusa_head.0.0.linear.weight', 'medusa_head.0.0.linear.bias', 'medusa_head.1.1.weight', 'medusa_head.3.1.weight', 'medusa_head.4.0.linear.bias', 'medusa_head.2.0.linear.weight', 'medusa_head.3.0.linear.bias

In [3]:
# KVキャッシュの初期化
past_key_values, past_key_values_data, current_length_data = initialize_past_key_values(model.base_model)
model.past_key_values = past_key_values
model.past_key_values_data = past_key_values_data
model.current_length_data = current_length_data

In [4]:
# プロンプトとトークナイズ確認
model.current_length_data.zero_() # 
prompt = "I stay with my dog named Retrieva. Do you have any dogs?"
input_ids = tokenizer([prompt]).input_ids
input_len = len(input_ids[0])

print("Prompt:", prompt)
print('Input token length:', len(input_ids[0]))

Prompt: I stay with my dog named Retrieva. Do you have any dogs?
Input token length: 16


In [5]:
with torch.inference_mode():
    model.current_length_data.zero_()
    output = model.base_model(torch.as_tensor(input_ids).cuda(), past_key_values=model.past_key_values,)
    print('output shape:', output.logits.shape)
    pred = output.logits.argmax(-1)
    input_ids[0] = input_ids[0] + pred[0, -1:].tolist()
    print('First prediction:', tokenizer.batch_decode(pred[0]))

output shape: torch.Size([1, 16, 32000])
First prediction: ['itel', "'", 'in', 'my', 'parents', ',', 'Max', 'ver', '.', 'She', 'you', 'have', 'any', 'p', '?', '\n']


In [6]:
with torch.inference_mode():
    for _ in tqdm(range(100)):
        output = model.base_model(pred[..., -1:], past_key_values=model.past_key_values, use_cache=True)
        pred = output.logits.argmax(-1)
        # pred_topk = output.logits.topk(10, dim = -1).indices[0]
        input_ids[0] = input_ids[0] + pred[0, -1:].tolist()
        #print(tokenizer.batch_decode(pred[..., -1:]))
        # print(tokenizer.batch_decode(pred_topk), 'topk:', pred_topk)

100%|██████████| 100/100 [00:10<00:00,  9.29it/s]


In [7]:
print(tokenizer.decode(input_ids[0]))

<s> I stay with my dog named Retrieva. Do you have any dogs?
Yes, I have a dog named Max. He'<s> The 2019-2024 Outlook for Non-Alcoholic Beverages in India
This study covers the latent demand outlook for non-alcoholic beverages across the states, union territories, and cities of India. Latent demand (in millions of U.S. dollars) or the number of units of non-alcoholic beverages is estimated
